## Неделя 2. Понедельник
### Обучение с учителем

### Применение базовых методов классификации

In [2]:
import pandas as pd
import numpy as np

In [3]:
# base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Важная настройка для корректной настройки pipeline!
import sklearn

sklearn.set_config(transform_output="default")

# Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    OrdinalEncoder,
    TargetEncoder,
)
from sklearn.model_selection import GridSearchCV, KFold

# for model learning
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    cross_val_score,
)

# models
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier

# from catboost import CatBoostRegressor

# Metrics
from sklearn.metrics import accuracy_score


# tunning hyperparamters model
# import optuna

#### 0. Ознакомьтесь с датасетом

In [4]:
df = pd.read_csv("data/heart.csv")

* __Age__: age of the patient [years]
* __Sex__: sex of the patient [M: Male, F: Female]
* __ChestPainType__: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
* __RestingBP__: resting blood pressure [mm Hg]
* __Cholesterol__: serum cholesterol [mm/dl]
* __FastingBS__: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
* __RestingECG__: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite * left ventricular hypertrophy by Estes' criteria]
* __MaxHR__: maximum heart rate achieved [Numeric value between 60 and 202]
* __ExerciseAngina__: exercise-induced angina [Y: Yes, N: No]
* __Oldpeak__: oldpeak = ST [Numeric value measured in depression]
* __ST_Slope__: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
* __HeartDisease__: output class [1: heart disease, 0: Normal]

* Таргетом является столбец `HeartDisease`. Необходимо предсказать по имеющимся данным, есть ли проблемы с сердцем

In [5]:
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40.0,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49.0,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37.0,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48.0,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54.0,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [6]:
df.dtypes

Age               float64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

#### 1. Небольшие рекомендации ниже 


* __Baseline pipeline (базовый пайплайн)__ - это простой пайплайн, который используется как отправная точка или точка сравнения при разработке и оценке более сложных моделей или алгоритмов. 

* Для этого сначала используйте самые простые идеи по заполнению пропусков(средними, медианами, модами) и кодированию категориальных данных, которые вам приходят в голову. 

* После того, как вы построите модели провалидируете их. Можно будет приступать к попыткам улучшить свою модель с помощью ваших идей - пробовать создавать новые фичи, кодировать данные по-другому, заполнять иначе NaN и тд

In [7]:
X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"]


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Размер X_train:", X_train.shape)
print("Размер X_test :", X_test.shape)
print("Размер y_train:", y_train.shape)
print("Размер y_test :", y_test.shape)

Размер X_train: (734, 11)
Размер X_test : (184, 11)
Размер y_train: (734,)
Размер y_test : (184,)


#### 2. Заполните пропущенные значения(`Imputing`), как считаете нужным.  

- Не забывайте памятку выше, сначала заполняйте самыми тривиальными идеями. Наприсер, средними, медианами и т.д

In [8]:
numeric_features = ["Age", "RestingBP", "Cholesterol", "FastingBS", "MaxHR", "Oldpeak"]
categorical_features = [
    "Sex",
    "ChestPainType",
    "RestingECG",
    "ExerciseAngina",
    "ST_Slope",
]

numeric_transformer = SimpleImputer(strategy="median")
categorical_transformer = SimpleImputer(strategy="most_frequent")

##### 2.1 Оберните в `ColumnTransformer` свой `Imputing` данных. Проверьте корректность его работы. Для этого необходимо сделать:

1. Обучить и трансформировать свой `Imputer` с помощью `your_imputer.fit_transform` - на тренировочных данных
2. Заполнить с помощью `your_imputer.transform` - на тестовых данных

Убедитесь, что данные прошли через этап `Imputing'а` и пропусков в них больше нет

In [9]:
my_imputer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [10]:
# 1. fit_transform на train
X_train_imputed = my_imputer.fit_transform(X_train)

# 2. transform на test
X_test_imputed = my_imputer.transform(X_test)

# 3. Проверка
# Для train
has_nan_train = np.any(pd.isna(X_train_imputed))
# Для test
has_nan_test = np.any(pd.isna(X_test_imputed))

print("Есть ли NaN в train?", has_nan_train)
print("Есть ли NaN в test? ", has_nan_test)

Есть ли NaN в train? False
Есть ли NaN в test?  False


#### 3. Закодируйте категориальные переменные, как считаете нужным

* `OneHotEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)  
* `TargetEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.TargetEncoder.html)  
* `OrdinalEncoding` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)  
* `CatBoostEncoding` (https://www.geeksforgeeks.org/categorical-encoding-with-catboost-encoder/)  

In [11]:
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

##### 3.1 Оберните в `ColumnTransformer` свой `Encoding` данных. Проверьте корректность его работы. 

In [12]:
my_encoder = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# 5. fit_transform на train
X_train_encoded = my_encoder.fit_transform(X_train)

# 6. transform на test
X_test_encoded = my_encoder.transform(X_test)

# 7. Проверка: выводим размеры и убеждаемся, что категориальные признаки закодировались
print("Размер X_train_encoded:", X_train_encoded.shape)
print("Размер X_test_encoded :", X_test_encoded.shape)
print("Тип X_train_encoded   :", type(X_train_encoded))

Размер X_train_encoded: (734, 20)
Размер X_test_encoded : (184, 20)
Тип X_train_encoded   : <class 'numpy.ndarray'>


In [13]:
print(X_test_encoded)

[[ 46. 115.   0. ...   0.   1.   0.]
 [ 58. 132. 224. ...   0.   0.   1.]
 [ 60. 125. 258. ...   0.   1.   0.]
 ...
 [ 49. 130.   0. ...   0.   1.   0.]
 [ 59. 110. 239. ...   0.   1.   0.]
 [ 57. 105.   0. ...   0.   1.   0.]]


#### 4. То же самое проделать с нормализацией данных

* `StandardScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* `MinMaxScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
* `RobustScaler` (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)

In [14]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

#### 4.1 Оберните в `ColumnTransformer` свой `Scaling` данных, проверьте корректность работы.

In [15]:
my_scaler = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

X_train_normalized = my_scaler.fit_transform(X_train)
X_test_normalized = my_scaler.transform(X_test)

print("Размер X_train_normalized:", X_train_normalized.shape)
print("Размер X_test_normalized :", X_test_normalized.shape)
print("Тип:", type(X_train_normalized))

Размер X_train_normalized: (734, 20)
Размер X_test_normalized : (184, 20)
Тип: <class 'numpy.ndarray'>


#### 5. Соберите весь препроцессинг в общий Pipeline.

In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

##### 5.1 Прогоните свои данные через `preprocessor` и убедитесь, что ваши данные проходят через него корректно и уже готовы к ML-модели

In [17]:
X_train_ready = preprocessor.fit_transform(X_train)
X_test_ready = preprocessor.transform(X_test)

print("Форма X_train после препроцессинга:", X_train_ready.shape)
print("Форма X_test после препроцессинга :", X_test_ready.shape)

Форма X_train после препроцессинга: (734, 20)
Форма X_test после препроцессинга : (184, 20)


#### 6.ML-модели

* `LogisticRegression` (из `sklearn.linear_model`)  
* `LogisticRegression with regularization` (из `sklearn.linear_model`)  
* `KNeighborsClassifier` (из `sklearn.neighbors`)  
* `DecisionTree` (из `sklearn.tree`)  

##### 6.1 Обучите свой `Pipeline` с помощью метода `.fit()` с разными моделями.

In [18]:
ml_pipeline_logreg = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", LogisticRegression(max_iter=1000, random_state=42)),
    ]
)

ml_pipeline_logreg.fit(X_train, y_train)

print("LogisticRegression")
print("Train score:", ml_pipeline_logreg.score(X_train, y_train))
print("Test score :", ml_pipeline_logreg.score(X_test, y_test))

LogisticRegression
Train score: 0.8596730245231607
Test score : 0.8858695652173914


In [19]:
logreg_train_predict = ml_pipeline_logreg.predict(X_train)
logreg_valid_predict = ml_pipeline_logreg.predict(X_test)

print("LogReg train preds (первые 10):", logreg_train_predict[:10])
print("LogReg valid preds (первые 10):", logreg_valid_predict[:10])

LogReg train preds (первые 10): [1 0 1 1 1 0 1 0 0 0]
LogReg valid preds (первые 10): [1 0 1 1 0 0 0 1 0 1]


In [20]:
ml_pipeline_logreg_l1 = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "model",
            LogisticRegression(
                penalty="l1", solver="liblinear", C=0.5, max_iter=1000, random_state=42
            ),
        ),
    ]
)

ml_pipeline_logreg_l1.fit(X_train, y_train)

print("\nLogisticRegression with regularization")
print("Train score:", ml_pipeline_logreg_l1.score(X_train, y_train))
print("Test score :", ml_pipeline_logreg_l1.score(X_test, y_test))


LogisticRegression with regularization
Train score: 0.8555858310626703
Test score : 0.8913043478260869


In [21]:
logreg_l1_train_predict = ml_pipeline_logreg_l1.predict(X_train)
logreg_l1_valid_predict = ml_pipeline_logreg_l1.predict(X_test)

print("LogReg+Reg L1 train preds (первые 10):", logreg_l1_train_predict[:10])
print("LogReg+Reg L1 valid preds (первые 10):", logreg_l1_valid_predict[:10])

LogReg+Reg L1 train preds (первые 10): [1 0 1 1 1 0 1 0 0 0]
LogReg+Reg L1 valid preds (первые 10): [1 0 1 1 0 0 0 1 0 1]


In [22]:
ml_pipeline_logreg_l2 = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "model",
            LogisticRegression(
                penalty="l2", solver="lbfgs", C=1.0, max_iter=1000, random_state=42
            ),
        ),
    ]
)

ml_pipeline_logreg_l2.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [23]:
logreg_l2_train_predict = ml_pipeline_logreg_l2.predict(X_train)
logreg_l2_valid_predict = ml_pipeline_logreg_l2.predict(X_test)

print("LogReg+L2 train preds (первые 10):", logreg_l2_train_predict[:10])
print("LogReg+L2 valid preds (первые 10):", logreg_l2_valid_predict[:10])

LogReg+L2 train preds (первые 10): [1 0 1 1 1 0 1 0 0 0]
LogReg+L2 valid preds (первые 10): [1 0 1 1 0 0 0 1 0 1]


In [24]:
ml_pipeline_knn = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", KNeighborsClassifier(n_neighbors=2)),
    ]
)

ml_pipeline_knn.fit(X_train, y_train)

print("\nKNeighborsClassifier (k=5)")
print("Train score:", ml_pipeline_knn.score(X_train, y_train))
print("Test score :", ml_pipeline_knn.score(X_test, y_test))


KNeighborsClassifier (k=5)
Train score: 0.9155313351498637
Test score : 0.8097826086956522


In [25]:
knn_train_predict = ml_pipeline_knn.predict(X_train)
knn_valid_predict = ml_pipeline_knn.predict(X_test)

print("KNN train preds (первые 10):", knn_train_predict[:10])
print("KNN valid preds (первые 10):", knn_valid_predict[:10])

KNN train preds (первые 10): [1 0 1 1 1 0 1 0 0 0]
KNN valid preds (первые 10): [1 0 1 0 0 0 0 1 0 1]


In [26]:
ml_pipeline_tree = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", DecisionTreeClassifier(random_state=42, max_depth=5)),
    ]
)

ml_pipeline_tree.fit(X_train, y_train)

print("\nDecisionTreeClassifier (max_depth=5)")
print("Train score:", ml_pipeline_tree.score(X_train, y_train))
print("Test score :", ml_pipeline_tree.score(X_test, y_test))


DecisionTreeClassifier (max_depth=5)
Train score: 0.8950953678474114
Test score : 0.7989130434782609


In [27]:
tree_train_predict = ml_pipeline_tree.predict(X_train)
tree_valid_predict = ml_pipeline_tree.predict(X_test)

print("Tree train preds (первые 10):", tree_train_predict[:10])
print("Tree valid preds (первые 10):", tree_valid_predict[:10])

Tree train preds (первые 10): [1 0 1 1 1 1 1 0 0 0]
Tree valid preds (первые 10): [1 1 1 0 1 0 0 0 0 1]


#### 7. С помощью метода `.predict()` (на вход поступают только матрица признаков, без целевой переменной) предсказать значения на обучающей выборке (`X_train`) и валидационной выборке (`X_valid`).

In [28]:
# model_train_predict
# model_valid_predict

"""Смотреть выше для каждой модели"""

'Смотреть выше для каждой модели'

##### 7.1 С помощью функции оценки качества (`accuracy_score`) собрать следующую таблицу ниже

* значение функции на обучающих данных
* значение функции на валидационных данных 
    
Результатом выполнения этого пункта будет `DataFrame` формата: 
    
|  |train|valid|
|--|-----|-----|
|**LogReg**|  train_score  | valid_score    |
|**LogReg with l1**|  train_score  | valid_score    |
|**LogReg with l2**|  train_score  | valid_score    |
|**KNN**| train_score  |  valid_score   |
|**Tree**| train_score | valid_score    |

In [29]:
# Словарь для результатов
results = {}

# 1. LogReg (по умолчанию = L2, solver='lbfgs')
results["LogReg"] = [
    accuracy_score(y_train, logreg_train_predict),
    accuracy_score(y_test, logreg_valid_predict),
]

# 2. LogReg with L1
results["LogReg with l1"] = [
    accuracy_score(y_train, logreg_reg_train_predict),
    accuracy_score(y_test, logreg_reg_valid_predict),
]

# 3. LogReg with L2 (явный блок)
results["LogReg with l2"] = [
    accuracy_score(y_train, logreg_l2_train_predict),
    accuracy_score(y_test, logreg_l2_valid_predict),
]

# 4. KNN
results["KNN"] = [
    accuracy_score(y_train, knn_train_predict),
    accuracy_score(y_test, knn_valid_predict),
]

# 5. Decision Tree
results["Tree"] = [
    accuracy_score(y_train, tree_train_predict),
    accuracy_score(y_test, tree_valid_predict),
]

# Формируем DataFrame
df_results = pd.DataFrame(results, index=["train", "valid"]).T
print(df_results)

NameError: name 'logreg_reg_train_predict' is not defined

In [ ]:
df_results

,train,valid
LogReg,0.859673,0.885870
LogReg with l1,0.855586,0.891304
LogReg with l2,0.859673,0.885870
KNN,0.915531,0.809783
Tree,1.000000,0.788043


#### 8. Теперь реализуйте __кросс-валидацию__ с KFold=5 и выведите средний __score__

In [31]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

# Определяем стратегию кросс-валидации
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Словарь моделей
models = {
    "LogReg": LogisticRegression(max_iter=1000, random_state=42),
    "LogReg with l1": LogisticRegression(
        penalty="l1", solver="liblinear", C=0.5, max_iter=1000, random_state=42
    ),
    "LogReg with l2": LogisticRegression(
        penalty="l2", solver="lbfgs", C=1.0, max_iter=1000, random_state=42
    ),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVC": SVC(kernel="linear", random_state=42),
    "Tree": DecisionTreeClassifier(max_depth=5, random_state=42),
}

# Словарь для результатов
cv_results = {}

# Прогоняем все модели через cross_val_score
for name, model in models.items():
    pipeline = Pipeline([("preprocessor", preprocessor), ("model", model)])

    scores = cross_val_score(pipeline, X, y, cv=cv, scoring="accuracy")
    cv_results[name] = np.mean(scores)

# Собираем в таблицу
df_cv_results = pd.DataFrame(cv_results, index=["cross_val_score"]).T
print(df_cv_results)

                cross_val_score
LogReg                 0.858351
LogReg with l1         0.859432
LogReg with l2         0.858351
KNN                    0.865966
SVC                    0.861606
Tree                   0.839813


In [ ]:
df_cv_results

,cross_val_score
LogReg,0.858351
LogReg with l1,0.859432
LogReg with l2,0.858351
KNN,0.865966
SVC,0.861606
Tree,0.839813


|  |cross_val_score|
|--|-----|
|**LogReg**|  your_score |
|**LogReg with l1**|  your_score  |
|**LogReg with l2**|  your_score  |
|**KNN**| your_score  |
|**SVC**| your_score  |
|**Tree**| your_score |

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Пора сохранить изменения для __github__. 

1. Перейди в командной строке в папку, в которой расположен этот нотбук. 
2. Выполни команду `git add 06-01-task.ipynb`
3. Выполни команду `git commit -m "base models in progress"`
4. Выполни команду `git push`

##### 9. Теперь, когда вы проделали весь pipeline и обучили базовую модель, можно вернуться к началу и пробовать новые идеи и искать точки роста для ваших моделей, в том числе и добавление новых фичей

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сохрани файл для __github__ и выполни команду `!git status` в ячейке ниже.


In [ ]:
# code